In [9]:
import sys, os 
import pandas as pd 

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer

In [47]:
def load_data():

    data = os.path.join("data", "train.csv")

    df = pd.read_csv(data)
    X_train = df[['id', 'comment_text']]
    y_train = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

    X_test = pd.read_csv(os.path.join("data", "test.csv"))
    y_test = pd.read_csv(os.path.join("data", "test_labels.csv"))
    test = X_test.merge(y_test, on='id')
    test = test[ (test['toxic']!=-1) | (test['severe_toxic']!=-1) | 
                (test['obscene']!=-1) | (test['threat']!=-1) | (test['insult']!=-1) 
                | (test['identity_hate']!=-1) ]
    test = test.reset_index(drop=True) 
    
    X_test = test[['id', 'comment_text']]
    y_test = test[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

    return X_train, y_train, X_test, y_test 

In [48]:
# Constants 

ngrams = 1
penalty = 'l2'
Cs = 10
cv = 5
max_iter = 1000 
scoring = 'accuracy'

In [49]:
X_train, y_train, X_test, y_test = load_data()

In [13]:
vectorizer = TfidfVectorizer(ngram_range=(1, ngrams))

In [14]:
def tfidf_transform(corpus, is_train=False):
    if is_train: 
        X = vectorizer.fit_transform(corpus)
    else:
        X = vectorizer.transform(corpus)
    return X

In [15]:
def print_results(model, X, y):
    """Print out evaluate a model, returns the metrics as tuple
    """
    prediction = model.predict(X)
    precision, recall, fbeta_score, support = \
        precision_recall_fscore_support(y, prediction)
    accuracy = accuracy_score(y, prediction)

    print ("Precision: {}\nRecall: {}\nF-Score: {}\nSupport: {}\nAccuracy {}\n".format(
            precision, recall, fbeta_score, support, accuracy))

    print (classification_report(y, prediction))

    print("Example predictions: ")
    print(prediction)

    return (precision, recall, fbeta_score, support, accuracy)

In [16]:
# turn the corpus into a list to pass into the vectorizer 
train_corpus = X_train['comment_text'].values.tolist()
test_corpus = X_test['comment_text'].values.tolist()

X_train_tfidf = tfidf_transform(train_corpus, is_train=True)
X_test_tfidf = tfidf_transform(test_corpus, is_train=False)

TypeError: string indices must be integers

In [ ]:
def run_model(): 
    labels = ['toxic' ] #, 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    
    for label in labels: 
        print("Train model on the \"{}\" label.\n".format(label))
        y_train_label = y_train[label].values.tolist()
        y_test_label = y_test[]
        
        lr = LogisticRegressionCV(   class_weight="balanced", 
                                     Cs = Cs, 
                                     cv = cv, 
                                     penalty = penalty, 
                                     scoring = scoring, 
                                     max_iter = max_iter)
        
        lr.fit(X_train_tfidf, y_train_label)
        
        print("Train Results\n")
        print_results(lr, X_train_tfidf, y_train_label)
        
        print("Test Results\n")
        
        print_results(lr, X_test_tfidf, y_test_label)